In [ ]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, ENf
# Reducers
from typing import Annotated
from langgraph.graph.message import add_messages

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]